In [1]:
import sys
import cv2
import numpy as np
from rasterio import Affine
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from transform.coordinates import get_affine_transform
from transform.coordinates import to_web_mercator
from extract.extract import get_coordinates
from extract.extract import read_image
from transform.geotiff import geotiff
from matplotlib import pyplot as plt
from rasterio.shutil import copy
import folium


In [2]:
IMAGE_PATH = '../../resources/us-army-1945.jpeg'
TEST_TIF_PATH = "/Users/viktor/Work/8gradplus/time-strolls/resources/tifs/test.tif"

# lat / lon convention!
LANDMARKS = {
    'Kirche St. Oswald': {         
        'pixel': (286, 2125),
        'gps': (48.619095, 14.030765)
    },
    'Laher Unteruresch': {
        'pixel': (863, 2610) , 
        'gps': (48.61017854015886, 14.04406485511563) ,
                },
    'Ruine Wittinghausen': {
        'pixel': (2919, 143),
        'gps': (48.64500581431426, 14.103290101060226),
              }, 
}

In [3]:
def show_rasterio(tif_path):
    with rasterio.open(tif_path) as src:
        fig, ax = plt.subplots()
        rasterio.plot.show(src, ax)


def show_tiles(path=TEST_TIF_PATH):
    """Needs locally running tile server (make run-api. in backend folder)"""
    
    tiles = f"http://127.0.0.1:8000/tiles/WebMercatorQuad/{'{z}'}/{'{x}'}/{'{y}'}.png?url=file:///{path}"
    center = [48.61017, 14.044]  # Example: Berlin, Germany
    
    m = folium.Map(
        location=center,
        zoom_start=16,
        tiles=None  # Disable default tiles
    )
    
    
    folium.TileLayer(
        tiles=tiles,
        attr='historic',
        name="1945",
    ).add_to(m)
    
    return m

In [4]:
import rasterio

def geotiff(image, A,  path, crs='EPSG:3857'):
    height, width, bands = image.shape
    # Bands at first place in gtiff
    image = image.transpose(2, 0, 1)

    config = dict(
        driver='GTiff',
        height=height,
        width=width,
        count=bands,
        dtype=image.dtype,
        crs=crs,
        transform=Affine(*A.flatten())
        )
    # Use MemoryFile to create in-memory GeoTIFF
    with rasterio.open(path, 'w', **config) as dataset:
        dataset.write(image)

In [5]:
pixel, gps = get_coordinates(LANDMARKS)
meters = to_web_mercator(gps)
A = get_affine_transform(pixel, meters)
geotiff(read_image(IMAGE_PATH), A, TEST_TIF_PATH)

In [6]:
#show_rasterio(TEST_TIF_PATH)
show_tiles(TEST_TIF_PATH)